<a href="https://colab.research.google.com/github/Danieljoshua720/Electricity-demand-forecasting-AIML/blob/main/GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam


In [ ]:
df=pd.read_csv('/content/powerdemand_5min_2021_to_2024_with weather.csv')

In [ ]:
df.head()

,Unnamed: 0,datetime,Power demand,temp,dwpt,rhum,wdir,wspd,pres,year,month,day,hour,minute,moving_avg_3
0,0,2021-01-01 00:30:00,2014.00,8.0,6.9,93.0,0.0,0.0,1017.0,2021,1,1.0,0.0,30.0,NaN
1,1,2021-01-01 00:35:00,2005.63,8.0,6.9,93.0,0.0,0.0,1017.0,2021,1,1.0,0.0,35.0,NaN
2,2,2021-01-01 00:40:00,1977.60,8.0,6.9,93.0,0.0,0.0,1017.0,2021,1,1.0,0.0,40.0,1999.076667
3,3,2021-01-01 00:45:00,1976.44,8.0,6.9,93.0,0.0,0.0,1017.0,2021,1,1.0,0.0,45.0,1986.556667
4,4,2021-01-01 00:50:00,1954.37,8.0,6.9,93.0,0.0,0.0,1017.0,2021,1,1.0,0.0,50.0,1969.470000


In [ ]:
df.tail()

,Unnamed: 0,datetime,Power demand,temp,dwpt,rhum,wdir,wspd,pres,year,month,day,hour,minute,moving_avg_3
232682,232682,2023-05-08 17:30:00,5576.40,34.8,13.6,28.0,90.0,16.6,1002.3,2023,5,8.0,17.0,30.0,5572.576667
232683,232683,2023-05-08 17:35:00,5545.02,34.8,13.6,28.0,90.0,16.6,1002.3,2023,5,8.0,17.0,35.0,5560.260000
232684,232684,2023-05-08 17:40:00,5520.15,34.8,13.6,28.0,90.0,16.6,1002.3,2023,5,8.0,17.0,40.0,5547.190000
232685,232685,2023-05-08 17:45:00,5510.07,34.8,13.6,28.0,90.0,16.6,1002.3,2023,5,8.0,17.0,45.0,5525.080000
232686,232686,2023-05-08 17:50:00,5458.91,34.8,13.6,28.0,90.0,16.6,1002.3,2023,5,NaN,NaN,NaN,NaN


In [ ]:

df.isnull()

,Unnamed: 0,datetime,Power demand,temp,dwpt,rhum,wdir,wspd,pres,year,month,day,hour,minute,moving_avg_3
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232682,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
232683,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
232684,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
232685,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
df=df.dropna()

In [ ]:
df.drop_duplicates()

,Unnamed: 0,datetime,Power demand,temp,dwpt,rhum,wdir,wspd,pres,year,month,day,hour,minute,moving_avg_3
2,2,2021-01-01 00:40:00,1977.60,8.0,6.9,93.0,0.0,0.0,1017.0,2021,1,1.0,0.0,40.0,1999.076667
3,3,2021-01-01 00:45:00,1976.44,8.0,6.9,93.0,0.0,0.0,1017.0,2021,1,1.0,0.0,45.0,1986.556667
4,4,2021-01-01 00:50:00,1954.37,8.0,6.9,93.0,0.0,0.0,1017.0,2021,1,1.0,0.0,50.0,1969.470000
5,5,2021-01-01 00:55:00,1925.21,8.0,6.9,93.0,0.0,0.0,1017.0,2021,1,1.0,0.0,55.0,1952.006667
6,6,2021-01-01 01:00:00,1907.98,8.0,6.9,93.0,0.0,0.0,1017.0,2021,1,1.0,1.0,0.0,1929.186667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232681,232681,2023-05-08 17:25:00,5559.36,34.0,14.0,30.0,70.0,13.0,1004.0,2023,5,8.0,17.0,25.0,5578.973333
232682,232682,2023-05-08 17:30:00,5576.40,34.8,13.6,28.0,90.0,16.6,1002.3,2023,5,8.0,17.0,30.0,5572.576667
232683,232683,2023-05-08 17:35:00,5545.02,34.8,13.6,28.0,90.0,16.6,1002.3,2023,5,8.0,17.0,35.0,5560.260000
232684,232684,2023-05-08 17:40:00,5520.15,34.8,13.6,28.0,90.0,16.6,1002.3,2023,5,8.0,17.0,40.0,5547.190000


In [ ]:
df = df.drop(columns=['Unnamed: 0'])


In [ ]:

num_cols = ['wdir', 'pres', 'year', 'month', 'day', 'hour', 'minute', 'moving_avg_3']
for col in num_cols:
    df[col] = df[col].fillna(df[col].median())


In [ ]:
df['datetime'] = pd.to_datetime(df['datetime'])


In [ ]:
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['hour'] = df['datetime'].dt.hour
df['minute'] = df['datetime'].dt.minute


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 232144 entries, 2 to 232685
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   datetime      232144 non-null  datetime64[ns]
 1   Power demand  232144 non-null  float64       
 2   temp          232144 non-null  float64       
 3   dwpt          232144 non-null  float64       
 4   rhum          232144 non-null  float64       
 5   wdir          232144 non-null  float64       
 6   wspd          232144 non-null  float64       
 7   pres          232144 non-null  float64       
 8   year          232144 non-null  int32         
 9   month         232144 non-null  int32         
 10  day           232144 non-null  int32         
 11  hour          232144 non-null  int32         
 12  minute        232144 non-null  int32         
 13  moving_avg_3  232144 non-null  float64       
dtypes: datetime64[ns](1), float64(8), int32(5)
memory usage: 22.1 MB


In [ ]:
X = df.drop(columns=['datetime', 'Power demand'])
y = df[['Power demand']]


In [ ]:
train_size = int(0.8 * len(df))
val_size   = int(0.1 * len(df))

X_train = X[:train_size]
X_val   = X[train_size:train_size + val_size]
X_test  = X[train_size + val_size:]

y_train = y[:train_size]
y_val   = y[train_size:train_size + val_size]
y_test  = y[train_size + val_size:]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler_X = MinMaxScaler(feature_range=(0, 1))

X_train_scaled = scaler_X.fit_transform(X_train)
X_val_scaled   = scaler_X.transform(X_val)
X_test_scaled  = scaler_X.transform(X_test)


In [ ]:
scaler_y = MinMaxScaler(feature_range=(0, 1))

y_train_scaled = scaler_y.fit_transform(y_train)
y_val_scaled   = scaler_y.transform(y_val)
y_test_scaled  = scaler_y.transform(y_test)


In [ ]:
print(y_train_scaled.shape)


(185715, 1)


In [ ]:
def create_sequences(X, y, n_past):
    Xs, ys = [], []
    for i in range(n_past, len(X)):
        Xs.append(X[i-n_past:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)


In [ ]:
n_past = 24  # example (24 hours / timesteps)

X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, n_past)
X_val_seq,   y_val_seq   = create_sequences(X_val_scaled,   y_val_scaled,   n_past)
X_test_seq,  y_test_seq  = create_sequences(X_test_scaled,  y_test_scaled,  n_past)


In [ ]:
def stacked_gru(input_shape):
    model = Sequential([
        GRU(128, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        GRU(64),
        Dense(1)
    ])
    model.compile(optimizer=Adam(0.001), loss='mse')
    return model


In [ ]:
def vanilla_gru(input_shape):
    model = Sequential([
        GRU(64, input_shape=input_shape),
        Dense(1)
    ])
    model.compile(optimizer=Adam(0.001), loss='mse')
    return model


In [ ]:
def bidirectional_gru(input_shape):
    model = Sequential([
        Bidirectional(GRU(64, return_sequences=True), input_shape=input_shape),
        GRU(32),
        Dense(1)
    ])
    model.compile(optimizer=Adam(0.001), loss='mse')
    return model


In [ ]:
def deep_gru(input_shape):
    model = Sequential([
        GRU(128, return_sequences=True, input_shape=input_shape),
        Dropout(0.3),
        GRU(64, return_sequences=True),
        Dropout(0.3),
        GRU(32),
        Dense(1)
    ])
    model.compile(optimizer=Adam(0.0005), loss='mse')
    return model


In [ ]:
def train_model(model, X_train, y_train, X_val, y_val, epochs=10):
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=512,
        verbose=1
    )
    return history


In [ ]:
def evaluate_model(model, X_test, y_test, scaler_y):
    y_pred_scaled = model.predict(X_test)

    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)

    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae  = mean_absolute_error(y_true, y_pred)
    r2   = r2_score(y_true, y_pred)

    return rmse, mae, r2


In [ ]:
models = {
    "Vanilla GRU": vanilla_gru,
    "Stacked GRU": stacked_gru,
    "Bidirectional GRU": bidirectional_gru,
    "Deep GRU": deep_gru
}

results = {}

for name, builder in models.items():
    print(f"\n🚀 Training {name}")
    tf.keras.backend.clear_session()

    model = builder(
        input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])
    )

    train_model(model, X_train_seq, y_train_seq, X_val_seq, y_val_seq)

    rmse, mae, r2 = evaluate_model(model, X_test_seq, y_test_seq, scaler_y)

    results[name] = {
        "RMSE": rmse,
        "MAE": mae,
        "R2": r2
    }



🚀 Training Vanilla GRU


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 43s 111ms/step - loss: 0.0135 - val_loss: 4.7526e-04
Epoch 2/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 39s 107ms/step - loss: 3.6121e-04 - val_loss: 5.4723e-04
Epoch 3/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 39s 107ms/step - loss: 2.9120e-04 - val_loss: 5.3343e-04
Epoch 4/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 40s 110ms/step - loss: 2.8937e-04 - val_loss: 3.7823e-04
Epoch 5/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 40s 109ms/step - loss: 2.3431e-04 - val_loss: 3.2968e-04
Epoch 6/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 41s 110ms/step - loss: 2.2908e-04 - val_loss: 3.6005e-04
Epoch 7/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 40s 110ms/step - loss: 2.1281e-04 - val_loss: 2.2375e-04
Epoch 8/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 40s 110ms/step - loss: 2.1331e-04 - val_loss: 2.3515e-04
Epoch 9/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 40s 110ms/step - loss: 1.9757e-04 - val_loss: 4.9516e-04
Epoch 10/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 40s 110ms/step - loss: 2.0215e-04 - val_loss: 2.4718e-04
725/725 ━━━━━━━━━━━━━━━━━━━━ 3s 4

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 151s 403ms/step - loss: 0.0138 - val_loss: 8.3961e-04
Epoch 2/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 203s 405ms/step - loss: 8.2287e-04 - val_loss: 4.3274e-04
Epoch 3/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 146s 404ms/step - loss: 5.8260e-04 - val_loss: 4.0904e-04
Epoch 4/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 145s 400ms/step - loss: 5.1345e-04 - val_loss: 6.8541e-04
Epoch 5/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 203s 404ms/step - loss: 4.5022e-04 - val_loss: 8.2433e-04
Epoch 6/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 146s 401ms/step - loss: 3.7729e-04 - val_loss: 0.0010
Epoch 7/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 145s 400ms/step - loss: 3.3193e-04 - val_loss: 6.2128e-04
Epoch 8/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 144s 397ms/step - loss: 3.0225e-04 - val_loss: 9.5683e-04
Epoch 9/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 203s 401ms/step - loss: 2.8099e-04 - val_loss: 3.6962e-04
Epoch 10/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 145s 401ms/step - loss: 2.6355e-04 - val_loss: 3.1456e-04
725/725 ━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 107s 280ms/step - loss: 0.0126 - val_loss: 5.2413e-04
Epoch 2/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 144s 286ms/step - loss: 3.4917e-04 - val_loss: 3.4967e-04
Epoch 3/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 103s 283ms/step - loss: 2.5828e-04 - val_loss: 2.3427e-04
Epoch 4/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 139s 276ms/step - loss: 2.4706e-04 - val_loss: 2.0976e-04
Epoch 5/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 142s 276ms/step - loss: 2.5179e-04 - val_loss: 1.8738e-04
Epoch 6/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 142s 275ms/step - loss: 2.2364e-04 - val_loss: 1.7586e-04
Epoch 7/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 101s 277ms/step - loss: 2.1108e-04 - val_loss: 2.6016e-04
Epoch 8/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 102s 281ms/step - loss: 2.0427e-04 - val_loss: 1.4228e-04
Epoch 9/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 100s 274ms/step - loss: 1.7618e-04 - val_loss: 1.5803e-04
Epoch 10/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 143s 277ms/step - loss: 1.9527e-04 - val_loss: 2.2751e-04
725/725 ━━━━━━━━━━━━━━━

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 192s 504ms/step - loss: 0.0242 - val_loss: 0.0010
Epoch 2/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 178s 491ms/step - loss: 0.0019 - val_loss: 8.5251e-04
Epoch 3/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 178s 491ms/step - loss: 0.0014 - val_loss: 7.4750e-04
Epoch 4/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 185s 508ms/step - loss: 0.0011 - val_loss: 5.1431e-04
Epoch 5/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 196s 492ms/step - loss: 8.8317e-04 - val_loss: 4.0252e-04
Epoch 6/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 179s 491ms/step - loss: 7.3470e-04 - val_loss: 3.9000e-04
Epoch 7/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 178s 491ms/step - loss: 6.7017e-04 - val_loss: 5.2884e-04
Epoch 8/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 178s 490ms/step - loss: 5.8070e-04 - val_loss: 3.6498e-04
Epoch 9/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 178s 489ms/step - loss: 5.7248e-04 - val_loss: 3.9213e-04
Epoch 10/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 178s 489ms/step - loss: 4.8986e-04 - val_loss: 3.3566e-04
725/725 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/s

In [ ]:
results_df = pd.DataFrame(results).T
print(results_df)


                         RMSE        MAE        R2
Vanilla GRU        105.990407  86.746059  0.989309
Stacked GRU        115.396332  96.308791  0.987327
Bidirectional GRU   95.828745  73.955316  0.991261
Deep GRU            91.564581  50.283220  0.992021


In [ ]:
results_df.to_csv("gru_results.csv")


In [ ]:
param_grid = {
    "units": [32, 64, 128],
    "dropout": [0.0, 0.2, 0.3],
    "learning_rate": [0.001, 0.0005],
    "batch_size": [32, 64],
    "epochs": [10]
}


In [ ]:
def build_gru_hp(input_shape, units, dropout, lr):
    model = Sequential([
        GRU(units, return_sequences=False, input_shape=input_shape),
        Dropout(dropout),
        Dense(1)
    ])

    model.compile(
        optimizer=Adam(learning_rate=lr),
        loss="mse"
    )
    return model


In [ ]:
results = []

for units in param_grid["units"]:
    for dropout in param_grid["dropout"]:
        for lr in param_grid["learning_rate"]:
            for batch_size in param_grid["batch_size"]:

                print(f"\n🚀 units={units}, dropout={dropout}, lr={lr}, batch={batch_size}")

                tf.keras.backend.clear_session()

                model = build_gru_hp(
                    input_shape=(X_train_seq.shape[1], X_train_seq.shape[2]),
                    units=units,
                    dropout=dropout,
                    lr=lr
                )

                model.fit(
                    X_train_seq, y_train_seq,
                    validation_data=(X_val_seq, y_val_seq),
                    epochs=20,
                    batch_size=batch_size,
                    verbose=0
                )

                val_pred = model.predict(X_val_seq)

                val_pred = scaler_y.inverse_transform(val_pred)
                val_true = scaler_y.inverse_transform(y_val_seq)

                rmse = np.sqrt(mean_squared_error(val_true, val_pred))

                results.append({
                    "units": units,
                    "dropout": dropout,
                    "lr": lr,
                    "batch_size": batch_size,
                    "val_RMSE": rmse
                })



🚀 units=32, dropout=0.0, lr=0.001, batch=32


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


725/725 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step

🚀 units=32, dropout=0.0, lr=0.001, batch=64


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


725/725 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step

🚀 units=32, dropout=0.0, lr=0.0005, batch=32


...

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


725/725 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step

🚀 units=32, dropout=0.3, lr=0.001, batch=64


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


725/725 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step

🚀 units=32, dropout=0.3, lr=0.0005, batch=32


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
grid_results = pd.DataFrame(results)
best_grid = grid_results.sort_values("val_RMSE").iloc[0]

print(best_grid)


In [ ]:
!pip install keras-tuner


In [ ]:
import keras_tuner as kt


In [ ]:
def gru_bayesian_model(hp):

    units = hp.Int("units", min_value=32, max_value=128, step=32)
    dropout = hp.Float("dropout", 0.0, 0.4, step=0.1)
    lr = hp.Choice("lr", [1e-3, 5e-4, 1e-4])

    model = Sequential([
        GRU(units, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])),
        Dropout(dropout),
        Dense(1)
    ])

    model.compile(
        optimizer=Adam(learning_rate=lr),
        loss="mse"
    )

    return model


In [ ]:
tuner = kt.BayesianOptimization(
    gru_bayesian_model,
    objective="val_loss",
    max_trials=15,
    directory="gru_tuning",
    project_name="bayesian_gru"
)


In [ ]:
tuner.search(
    X_train_seq,
    y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=20,
    batch_size=64,
    verbose=1
)


In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]


In [ ]:
y_test_pred_scaled = best_model.predict(X_test_seq)

y_test_pred = scaler_y.inverse_transform(y_test_pred_scaled)
y_test_true = scaler_y.inverse_transform(y_test_seq)

rmse = np.sqrt(mean_squared_error(y_test_true, y_test_pred))
mae  = mean_absolute_error(y_test_true, y_test_pred)
r2   = r2_score(y_test_true, y_test_pred)

print("RMSE:", rmse)
print("MAE :", mae)
print("R2  :", r2)


In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel("Epochs")
plt.ylabel("MSE Loss")
plt.title("GRU Training vs Validation Loss")
plt.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel("Epochs")
plt.ylabel("MSE Loss")
plt.title("GRU Training vs Validation Loss")
plt.legend()
plt.show()


In [ ]:
pivot = grid_results.pivot_table(
    values="val_RMSE",
    index="units",
    columns="batch_size"
)

plt.figure()
plt.imshow(pivot, aspect='auto')
plt.colorbar(label="Validation RMSE")
plt.xticks(range(len(pivot.columns)), pivot.columns)
plt.yticks(range(len(pivot.index)), pivot.index)
plt.xlabel("Batch Size")
plt.ylabel("GRU Units")
plt.title("Grid Search RMSE Heatmap")
plt.show()


In [ ]:
val_losses = [t.metrics.get_last_value("val_loss")
              for t in tuner.oracle.trials.values()]

plt.figure()
plt.plot(val_losses, marker='o')
plt.xlabel("Trial Number")
plt.ylabel("Validation Loss (MSE)")
plt.title("Bayesian Optimization Progress")
plt.show()


In [ ]:
plt.figure()
plt.plot(y_test_true[:200], label="Actual")
plt.plot(y_test_pred[:200], label="Predicted")
plt.xlabel("Time Steps")
plt.ylabel("Target Value")
plt.title("GRU Prediction vs Actual")
plt.legend()
plt.show()


In [ ]:
errors = y_test_true - y_test_pred

plt.figure()
plt.hist(errors, bins=30)
plt.xlabel("Prediction Error")
plt.ylabel("Frequency")
plt.title("Error Distribution (GRU)")
plt.show()


In [ ]:
results_df[['RMSE']].plot(kind='bar')
plt.xlabel("Model")
plt.ylabel("RMSE")
plt.title("GRU Variants Performance Comparison")
plt.show()
